# Magnetic Dipole App

Now that we have learned how to interact with `geoh5`, we can start implementing our first application. We are going to create a magnetic dipole simulator from scratch, which could become a useful tool to interpret magnetic data maps.


![mag_dipole_gif](./images/mag_dipole_app.gif "segment")

## Background

Python needs to import external libraries for certain classes and functions to be recognized when running the code. That is done below.

In [1]:
# Import the necessary packages
from __future__ import annotations

import sys
import numpy as np

from geoh5py.data import Data
from geoh5py.objects import ObjectBase, Points
from geoh5py.ui_json import InputFile, constants, templates
from geoh5py.ui_json.utils import monitored_directory_copy
from geoh5py.workspace import Workspace

### Arrays
We will make use a few useful methods on `numpy` arrays.
- The `sum()` method applied to a specific axis of an array, collapsing the dimension.
- The `dot()` method performs the dot product between two arrays
- The transpose (`.T`) method to get the dimensions of arrays to align:

$[1 \times 3] \cdot [M \times 3].T \rightarrow [1 \times M] $.

- Array broadcasting (repeat) of a 1D array along a second dimension using the `[:, None]` indexing:

$[N \times 3] \;/\;([1 \times N][: None]) \rightarrow [N \times 3] $

## The physics

The strength of the magnetic field flux of a single dipole is:

$$\mathbf{B} = \frac{\mu_0}{4 \pi} \left( \frac{3 \mathbf{r} (\mathbf{m} \cdot \mathbf{r})}{r^5} - \frac{\mathbf{m}}{r^3} \right)$$

where $\mu_0$ is a constant ($4 \pi \times 10^{-7}$), $\mathbf{m}$ is the magnetic dipole moment (susceptibility $\times$ volume), and $\mathbf{r}$ is the vector between the dipole and the locations.

In a geophysical context however, we normally get Total Magnetic Intensity (TMI) data from magnetometers. The conversion can be approximated by a projection of the fields onto the direction of Earth's inducing field.

$$
b_{TMI} \approx \mathbf{\hat H}_0 \cdot \mathbf{b}
$$

So we will need at least to
 - Create functions for:
    - Computing the fields of a dipole(s)
    - Converting vector fields to TMI.
 - Provide inputs for parameters:
    - Receivers (observation) locations
    - Source dipole locations
    - Dipole moment values
    - Dipole inclination angles
    - Dipole declination angles
    - Earth's field inclination
    - Earth's field declination

## Functions

### Dipole function

We can begin with writing the function needed to compute the
magnetic field from a single dipole. The function should be able to take as argument

    - Coordinates of the source and receivers (defining 'r'),
    - Parameters defining the strength (moment) and orientation (inc, dec) of the dipole source.

### Inclination to declination conversion
We require a function to convert inclination/declination angles to Cartesian vectors. The same function will come handy later when computing the TMI projection.

In [3]:
def inclination_declination_2_xyz(inclination, declination):
    """Convert inclination and declination angles (degrees) to unit vector (xyz)."""
    theta = np.deg2rad((90 - declination) % 360)
    phi = np.deg2rad(90 + inclination)
    xyz = np.c_[np.sin(phi) * np.cos(theta), np.sin(phi) * np.sin(theta), np.cos(phi)]

    return xyz

We can now test our `b_field` function with existing objects present in our `geoh5` project created and saved at the beginning of the workshop.

The `b_field` function returns an array for the three components of the
magnetic field due to the dipole source.

Let's write a simple function to get the TMI (projected) data instead.

and test it for the inducing field parameters at the test site (single location):

## The Simulator (creating a generic function)

In this section, we are going to string together the functions defined above and generalize our approach so that we can deal with different types of `geoh5` objects. Let's wrap all the previous functions into a `magnetic_simulator` class.

We now have a generic function to compute TMI data based on almost any type of ANALYST object.

Let us edit our Points object to add more vertices and create data arrays for moment, azimuth and dips. We can now run the simulation by simply giving those entities to our class.

![bfield](./images/b_field.png)

## The Driver (Running with Geoscience ANALYST)

In the previous section, we created our simulation program called `magnetic_simulator` that could compute the fields of dipoles defined by `geoh5` object. As a final step, we need to define a `driver` program that can be called from ANALYST.

We are going to take advantage of the
[geoh5py.ui_json.InputFile](https://geoh5py.readthedocs.io/en/stable/content/api/geoh5py.ui_json.html#geoh5py.ui_json.input_file.InputFile)
class to handle value conversions between geoh5 and python, as well as the `monitored_directory_copy` function to handle the live link with ANALYST.

The final step requires to make this program available to python to be executed from ANALYST. That requires going from a notebook to a .py file. This is done by:

- Use the `File\Download as\Python *.py` dropdown menu to convert this notebook to a `py` file.
- Rename the file to `mag_dipole_app.py`.
- Remove everything in the file except the
    - the imports,
    - the blocks of functions `def`
    - the following lines (without #'s and $\textbf{no}$ indentation)

    ```
    if __name__ == "__main__":
        file = sys.argv[1]
        run(file)
    ```

This part becomes the entry point of the python interpreter when running the
`mag_dipole_app.py` script.

## The User-Interface (ui.json)

In the previous section covering the [geoh5 API](geoh5-API), we have introduced functionality to compute the magnetic field from dipole locations. We are now going to create a user-interface (UI) to be able to run this program from a Geoscience ANALYST session.

We now can create our ui.json to provide inputs for the following 7 parameters:

- `Object` defining the dipoles (sources)
- `Object` defining the receivers
- `Data` or `float` value for the dipole moments
- `Data` or `float` value for the dipole inclination angles
- `Data` or `float` value for the dipole declination angles
- `Float` value for Earth's field inclination angle
- `Float` value for Earth's field declination angle

Let's create a UI.json with forms for each one of those inputs. Keep the dipoles as points only: {202C5DB1-A56D-4004-9CAD-BAAFD8899406}

Internally, when executed from ANALYST, this is what is going to happen

```
conda activate python-training
python mag_dipole [uijson]
```

Then your mag_dipole.run should be able to do the rest: compute and store the result.

Let's write out our ui.json to file.

 Copyright (c) 2022 Mira Geoscience Ltd.